In [1]:
import pandas
import numpy
import math

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    km = 6367 * c
    return km

In [2]:
df0 = pandas.read_csv('../../../Xharecost_MS_annex/hour9.csv',usecols= ['pickup_datetime','passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'])

In [8]:
df = df0[(abs(df0.pickup_latitude-0) > 1e-5) & \
         (abs(df0.pickup_longitude-0) > 1e-5) & \
         (abs(df0.dropoff_latitude-0) > 1e-5) & \
         (abs(df0.dropoff_longitude-0) > 1e-5)]

In [9]:
df.describe()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
count,12651.000000,12651.000000,12651.000000,12651.000000,12651.000000
mean,1.100229,-73.977511,40.754260,-73.978671,40.752809
std,0.368836,0.026140,0.023441,0.027182,0.023173
min,1.000000,-74.017921,40.616276,-74.295082,40.585709
25%,1.000000,-73.992001,40.740887,-73.991848,40.741989
50%,1.000000,-73.981102,40.755630,-73.980965,40.754936
75%,1.000000,-73.966015,40.770000,-73.971169,40.764204
max,4.000000,-73.697060,40.865898,-73.697060,40.964172


In [2]:
a = numpy.zeros((2,2))

In [5]:
a[0] = numpy.asarray([1,1])
print a

[[ 1.  1.]
 [ 0.  0.]]
